In [6]:
import pandas as pd
from surprise.prediction_algorithms.random_pred import NormalPredictor
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise.prediction_algorithms.slope_one import SlopeOne
import io

In [20]:
pd.DataFrame(data={'Brewery': ['Brouwerij Westvleteren','The Alchemist','Kern River Brewing Company','Brouwerij Drie Fonteinen','Russian River Brewing Company'],
                   'Beer': ['Trappist Westvleteren 12','Heady Topper','Citra DIPA','Armand 4 Oude Geuze Lente','Pliny The Elder']})

,Brewery,Beer
0,Brouwerij Westvleteren,Trappist Westvleteren 12
1,The Alchemist,Heady Topper
2,Kern River Brewing Company,Citra DIPA
3,Brouwerij Drie Fonteinen,Armand 4 Oude Geuze Lente
4,Russian River Brewing Company,Pliny The Elder


In [5]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 16.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617765 sha256=52d663d0412526e368f4a23641c9bd222814891cd6834fbed4d3b32dc552a392
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [1]:
def load_data():
    import pandas as pd

    beers_key = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beers_key.csv', index_col=0)
    prediction_table = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/beer_ids_for_prediction.csv', index_col=0)
    test = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/test.csv', index_col=0)
    train = pd.read_csv('https://raw.githubusercontent.com/kevin-o-lewis/beer-recommender/main/data/train.csv', index_col=0)

    return beers_key,prediction_table,test,train

In [2]:
def fit_to_trainset(algo):
    from surprise import Reader
    
    reader = Reader(rating_scale=(1, 10))
    suprise_train = Dataset.load_from_df(train, reader)
    full_trainset = suprise_train.build_full_trainset()
    algo = algo.fit(full_trainset)
    print('Fit complete.')
    return algo

In [40]:
def recommend(algo, user):
    '''
    Parameters:
        pre-fit model
        username
    Returns:
        Top 5 recommended beers for that user
    '''
    if train['review_profilename'].value_counts()[user] < 10:
        print('This user has too few reviews to make a personalized recommendation. Try our most popular beers instead!')
        return pd.DataFrame(data={'Brewery': ['Brouwerij Westvleteren','The Alchemist','Kern River Brewing Company','Brouwerij Drie Fonteinen','Russian River Brewing Company'],
                   'Beer': ['Trappist Westvleteren 12','Heady Topper','Citra DIPA','Armand 4 Oude Geuze Lente','Pliny The Elder']})

    prediction_table['user'] = user
    
    for idx in prediction_table.index:
        prediction_table.loc[idx, ('predicted_rating')] = algo.predict(user, prediction_table['beer_id'][idx])[3]
    top_5 = prediction_table.sort_values(by='predicted_rating', ascending=False)[0:5]['beer_id']
    results = pd.DataFrame()
    for beer_id in top_5:
        results = results.append(beers_key[beers_key['beer_beerid']==beer_id])


    return results

In [3]:
beers_key,prediction_table,test,train = load_data()

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
model1 = NormalPredictor()
model2 = SVD(n_factors= 10, n_epochs= 20, lr_all= 0.005, reg_all= 0.03)

In [11]:
fit_to_trainset(model1)
fit_to_trainset(model2)

Fit complete.
Fit complete.


In [41]:
recommend(model1, 'changeup45')

,beer_beerid,beer_name,brewery_name
418664,50543,Black Science,AMB - Maître Brasseur
1233034,10449,Railhead Red,Beaver Street Brewery & Whistle Stop Cafe
1008975,52011,4:20 Smuggle Brew,Brouwerij Kerkom
934063,17234,Karlsbräu,Karlsberg Brauerei
97423,23083,Hvede,Herslev Bryghus


In [39]:
user_test_ratings = test[test['review_profilename']=='changeup45']
user_test_ratings[user_test_ratings['beer_beerid']==39839]

,review_profilename,beer_beerid,review_overall
